In [1]:
import codecs
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
data_dir = "data/input/*.out"
paths = [p for p in glob.glob(data_dir)]
initial_key = "low   system:  model energy"
end_key = "ONIOM: calculating electric field derivatives."
result_dir = "data/output/result.csv"
df_output = pd.DataFrame()

In [3]:
for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_keyを含む行を探す
        initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
        initial_key_index = initial_key_df.index[0]
        df = df.iloc[initial_key_index:]
        # initial_keyを含む行以降のend_keyを含む行を抜き出す
        end_key_df = df[df['Col_1'].str.contains(end_key)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_index = end_key_df.index[0] - initial_key_index
        df = df.iloc[:end_key_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df['Col_1'])[0].split(":")[-1])
        high_model = float(list(df['Col_1'])[1].split(":")[-1])
        low_real = float(list(df['Col_1'])[2].split(":")[-1])
        high_real = float(list(df['Col_1'])[3].split("=")[-1])
        
        df_E = pd.DataFrame(data=[high_real, low_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["high real", "low real", "high model", "low model"]).T
        df_output = pd.concat([df_output, df_E])
         

In [4]:
df_output = df_output.sort_index()

In [5]:
df_output.to_csv(result_dir)